In [1]:
import pandas as pd
import os
import re
import numpy as np
from collections import Counter
import sys
import gc
sys.path.append('../')
import helper

In [2]:
csv_lis = []
for folder in os.listdir():
    if os.path.isdir(folder):
        for csv in os.listdir(folder):
            if (os.path.isfile(folder+'/'+csv)
                and csv.endswith('.csv')):
                csv_lis.append(folder+'/'+csv)

In [3]:
ctr = Counter([i.split('/')[0] for i in csv_lis])

df_dict = {}
for k in ctr:
    df_dict[k] = []

for csv in csv_lis:
    prf = csv.split('/')[0]
    df_dict[prf].append(pd.read_csv(csv))

In [4]:
df_text = {}
for k in df_dict:
    df_text[k] = []

for k, v in df_dict.items():
    for df in v:
        for t in df.texto:
            df_text[k].append(t)

df_filtered_text = {}
for k in df_dict:
    df_filtered_text[k] = []

for k, v in df_dict.items():
    for df in v:
        for t in df.texto:
            if len(t) < 1000:         
                df_filtered_text[k].append(t)


df_big_text = {}
for k in df_dict:
    df_big_text[k] = []

for k, v in df_dict.items():
    for df in v:
        for t in df.texto:
            if len(t) > 1000:         
                df_big_text[k].append(t)



In [5]:
whole = [(len(df_text[k]), k) for k in df_text]
lt = [(len(df_filtered_text[k]), k) for k in df_filtered_text]
gt = [(len(df_big_text[k]), k) for k in df_big_text]
print('WHOLE TEXT:')
for sz, key in sorted(whole, reverse=True):
    print(f'\t{sz} samples of {key}')

# print("LESS THAN 1000 text:")
# for sz, key in sorted(lt, reverse=True):
#     print(f'\t{sz} samples of {key}')
# print("GREATER THAN 1000 text:")
# for sz, key in sorted(gt, reverse=True):
#     print(f'\t{sz} samples of {key}')

del whole, lt, gt
gc.collect()

WHOLE TEXT:
	605 samples of Substituicao
	118 samples of Retirements
	61 samples of Exoneracao
	49 samples of NomeacaoComissionados
	25 samples of SemEfeitoAposentadoria
	10 samples of Cessoes
	9 samples of AbonoPermanencia
	2 samples of RetAposentadoria
	1 samples of Revertions


20

## A maior quantidade de casos extraídos é "subtituição". Porém, não foi possível realizar a curadoria das extrações. Por esta razão, serão utilizados os resultados de "semEfeitoAposentadoria"

In [9]:
sent_dict = {}
for k in df_text:
    sent_dict[k] = []
    for s in df_text[k]:
        sent_dict[k].extend(helper.sentencize_dodf(helper.preprocess(s)))
print("sentence number:")
for k, v in sorted(sent_dict.items(), key=lambda x:len(x[1]), reverse=True):
    print('\t',k, ':', len(v))

sentence number:
	 Substituicao : 605
	 Retirements : 304
	 Exoneracao : 61
	 Cessoes : 54
	 NomeacaoComissionados : 49
	 SemEfeitoAposentadoria : 40
	 AbonoPermanencia : 12
	 RetAposentadoria : 4
	 Revertions : 2


In [10]:
for ret in sent_dict['SemEfeitoAposentadoria']:
    print(ret, end=3*'\n')

TORNAR SEM EFEITO A RETIFICACAO DA AVERBACAO DE TEMPO DE SERVICO da servidora LUZIA SOUZA DOS PASSOS, matricula 01362518, AUXILIAR DE ENFERMAGEM, Secretaria de Estado de Saude, publicada no DODF No 243, quinta-feira, 21 de dezembro de 2017, pag. 42. .


TORNAR SEM EFEITO, na Ordem de Servico no 021, de 22 janeiro de 2020, publicada no DODF no 017, de 24/01/2020, pagina 24, o ato queRETIFICOUa AVERBACAO DE TEMPO DE SERVICO da servidora ANTONIA MARIA DA CONCEICAO, matricula no 135.843-X, AUXILIAR DE ENFERMAGEM, lotada no HMIB/SES, de 4.886 dias, ou seja, 13 anos, 4 meses e 21 dias, conforme certidao expedida pelo INSS, nos periodos de 01/09/1981 a 01/01/1984, de 15/03/1984 a 31/12/1984, de 14/01/1985 a 13/06/1987 e de 27/07/1989 a 31/05/1995, contados somente para fins de aposentadoria, de acordo com o processo no 272.000.401/2015. .


TORNAR SEM EFEITO, na Ordem de Servico no 021, de 22 janeiro de 2020, publicada no DODF no 017, de 24/01/2020, pagina 24, o ato queRETIFICOUa AVERBACAO DE

## Experimentação com xgboost (bons resultados previamente obtidos)

In [51]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_validate

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import LabelEncoder

## IDEIA: supor todas as frases de um DODF n pertencentes da atos, ver resultados(podre, mas sem dados rotulados complica)

In [109]:
prf = '../pdf/results/txt/'
fname = prf + '02-04-2019_canvas.txt'
raw = open(fname).read()
sents = helper.sentencize_dodf(raw)
sents = [i for i in sents if i != i.upper()]
sents = [i for i in sents if '...' not in i]
for s in sents:
    print(s, end=3*'\n')

ANO XLVIII EDICAO No- 62
BRASILIA - DF, TERCA-FEIRA, 2 DE ABRIL DE 2019 SECAO I SUMARIO
SECAO I
PAG.


PROCESSO: 001-000064/2018 - Volume 13 - Interessado: CLINUS - Especialidades Odontologicas
LTDA., valor R$ 2.169,44 (dois mil cento e sessenta e nove reais e quarenta e quatro centavos), referente
a nota fiscal no 841.


PROCESSO: 001-000066/2018 - Volume 13 - Interessado: ALCANCE, valor R$ 782,67 (setecentos e
oitenta e dois reais e sessenta e sete centavos), referente a nota fiscal no 2.755.


PROCESSO: 001-000086/2018 - Volume 10 - Interessado: IOE - INSTITUTO DE ODON TO L O G I A
ESPECIALIZADA, valor R$ 667,78 (seiscentos e sessenta e sete reais e setenta e oito centavos), referente
a nota fiscal no 383.


PROCESSO: 001-000100/2018 - Volume 15 - Interessado: VIVER - CLINICA DE IMAGENS MEDICAS
LTDA., valor R$ 5.371,60 (cinco mil trezentos e setenta e um reais e sessenta centavos), referente a nota
fiscal no 2.156.


PROCESSO: 001-000120/2018 - Volume 5 - Interessado: CLINICA SANTE 

In [110]:
df_sea = pd.DataFrame()

# not_sea = {'X': sents, 'y': len(sents) * [0]}
not_sea = {'X': [], 'y': []}

for s in sents:
    not_sea['X'].append(s)
    if 'PROCESSO' in s or re.search(r'\bresolve\b', s):
        not_sea['y'].append(1)
    else:
        not_sea['y'].append(0)
        

for k in df_text:
    for t in sent_dict[k]:
        not_sea['X'].append(t)
        not_sea['y'].append(1)

df_sea = pd.DataFrame(not_sea)
df_sea.head(), df_sea.y.value_counts()

(                                                   X  y
 0  ANO XLVIII EDICAO No- 62\nBRASILIA - DF, TERCA...  0
 1  PROCESSO: 001-000064/2018 - Volume 13 - Intere...  1
 2  PROCESSO: 001-000066/2018 - Volume 13 - Intere...  1
 3  PROCESSO: 001-000086/2018 - Volume 10 - Intere...  1
 4  PROCESSO: 001-000100/2018 - Volume 15 - Intere...  1,
 0    1488
 1    1395
 Name: y, dtype: int64)

In [111]:
# label transformer (One vs ALL)
# df_sea['y'] = df_sea.y.apply(
#     lambda x: 1 if x == 'SemEfeitoAposentadoria' else 0)

# df_sea['y'] = LabelEncoder().fit_transform(df_sea.y)
# df_sea.y = 1
df_sea.y.value_counts()

0    1488
1    1395
Name: y, dtype: int64

In [112]:
X, y = df_sea.X, df_sea.y
pipe = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('xgb',xgb.XGBClassifier(
        objective='multi:softprob',
#         num_class=len(df_text),        
        num_class=2,
    ))
])

In [113]:
%%time
cv_macro = cross_validate(pipe, X, y, scoring='f1_macro')
cv_micro = cross_validate(pipe, X, y, scoring='f1_micro')
cv_weighted = cross_validate(pipe, X, y, scoring='f1_weighted')
cv_accuracy = cross_validate(pipe, X, y, scoring='accuracy')

CPU times: user 2min 54s, sys: 160 ms, total: 2min 54s
Wall time: 24.4 s


In [114]:
print('cv_macro:', cv_macro['test_score'])
print('cv_micro:', cv_micro['test_score'])
print('cv_weighted:', cv_weighted['test_score'])
print('cv_accuracy:', cv_accuracy['test_score'])


cv_macro: [0.51071718 0.9323692  0.58842292 0.77461614 0.6332162 ]
cv_micro: [0.59965338 0.93240901 0.63258232 0.78298611 0.671875  ]
cv_weighted: [0.51758624 0.93231516 0.58398362 0.77597344 0.63693737]
cv_accuracy: [0.59965338 0.93240901 0.63258232 0.78298611 0.671875  ]
